In [1]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet
import pymannkendall as mk
from sktime.datasets import load_airline
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.theta import ThetaForecaster
from sktime.utils.plotting import plot_series

In [2]:
well_data = pd.read_excel('../data/Data-Wells.xlsx')

well_data.head(10)

KeyboardInterrupt: 

In [ ]:
well_data.head(10)

rm_nd = well_data.loc[well_data['1,4-Dioxane Results (ppb)'] != 'nd'].copy()

rm_capital_nd = rm_nd.loc[rm_nd['1,4-Dioxane Results (ppb)'] != 'ND'].copy()

rm_capital_nd['1,4-Dioxane Results (ppb)'] = rm_capital_nd['1,4-Dioxane Results (ppb)'].map(lambda x: x.replace(',', ''))

rm_capital_nd['1,4-Dioxane Results (ppb)'] = rm_capital_nd['1,4-Dioxane Results (ppb)'].map(lambda x: x.replace('<', ''))

res = rm_capital_nd
res.loc[:, 'ds']= pd.to_datetime(res['Date Sampled'])
res.loc[:, 'dioxane_results']= pd.to_numeric(res['1,4-Dioxane Results (ppb)'])
res.head(5)

In [ ]:
di_time = res.iloc[:, lambda df: df.columns.str.contains('Well Name|ds|dioxane_results',
                                              case=False)].copy()
di_time.rename(columns = {'dioxane_results':'y'}, inplace = True)
di_time.head(20)

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly
import plotly.offline as py
import plotly.graph_objs as go

IS_DEBUG = False

py.init_notebook_mode()

grouped = di_time.groupby('Well Name')
count = 1
n_rows = 2
n_cols = 1
fig = plt.figure(figsize=(5*n_cols, 5*n_rows))

mk_res = []

print(len(grouped))

for name, group in grouped:
    # if count > n_rows:
    #     break
    if len(group) < 150:
        continue

    max_value = group['y'].max()
    if IS_DEBUG:
        print(max_value)
        print(name)
        print(group)
    test = group

    test['cap'] = max_value
    test['floor'] = 0
    model = Prophet(growth = 'logistic')
    # fit the model
    model.fit(test)

    future = model.make_future_dataframe(periods=1826)
    future['cap'] = max_value
    future['floor'] = 0

    # use the model to make a forecast
    forecast = model.predict(future)
    # summarize the forecast
    if IS_DEBUG:
        print("forecast:")
        print(forecast[['ds', 'yhat']].head())
    # plot forecast
    # model.plot(forecast)

    fig = plot_plotly(model, forecast)  # This returns a plotly Figure
    py.iplot(fig)

    # Data generation for analysis
    data = test.y
    trend, h, p, z, Tau, s, var_s, slope, intercept = mk.original_test(data)

    mk_res.append([name, trend, h, p, z, Tau, s, var_s, slope, intercept])

    count+=1


mk_df = pd.DataFrame(mk_res, columns=['Well name', 'trend', 'h', 'p', 'z', 'Tau', 's', 'var_s','slope' , 'intercept'])
mk_df

In [ ]:

y = load_airline()
print(y)
y_train, y_test = temporal_train_test_split(y)
plot_series(y_train, y_test, labels=["y_train", "y_test"])
fh = ForecastingHorizon(y_test.index, is_relative=False)
forecaster = ThetaForecaster(sp=12)  # monthly seasonal periodicity
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
type(y)

In [ ]:
from sktime.forecasting.compose import (
    EnsembleForecaster,
    TransformedTargetForecaster,
    DirectRegressionForecaster,
    ReducedForecaster
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sklearn.neighbors import KNeighborsRegressor
from sktime.forecasting.online_learning import (
    NormalHedgeEnsemble,
    OnlineEnsembleForecaster,
)
max_value = di_time["y"].max()
# set_index_dt = di_time.set_index(pd.DatetimeIndex(di_time['ds']))

grouped = di_time.groupby('Well Name')
count = 1
n_rows = 2
n_cols = 1
fig = plt.figure(figsize=(5*n_cols, 5*n_rows))

mk_res = []

for name, group in grouped:
    if count > n_rows:
        break
    if len(group) < 10:
        continue

    print(group)
    test = group.set_index('ds')['y'].rename_axis(None)
    test.index = test.index.to_period("M")
    print(test)
    print('test',type(test))
    y_train, y_test = temporal_train_test_split(test)
    print(type(y_test.index))


    ses = ExponentialSmoothing(sp=12)
    holt = ExponentialSmoothing(trend="mul", damped_trend=False, sp=12)
    damped = ExponentialSmoothing(trend="mul", damped_trend=True, sp=12)

    hedge_expert = NormalHedgeEnsemble(n_estimators=3)
    forecaster = OnlineEnsembleForecaster(
    [
        ("ses", ses),
        ("holt", holt),
        ("damped", damped),
    ],
    ensemble_algorithm=hedge_expert,
    )

    forecaster.fit(y_train)
    y_pred = forecaster.update_predict(y_test)
    plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    count+=1

In [ ]:
a = 1
a

In [ ]:

from fbprophet.plot import plot_plotly, plot_components_plotly
import plotly.offline as py
import plotly.graph_objs as go

IS_DEBUG = False

py.init_notebook_mode()

grouped = di_time.groupby('Well Name')
count = 1
n_rows = 2
n_cols = 1
fig = plt.figure(figsize=(5*n_cols, 5*n_rows))

mk_res = []

print(len(grouped))

for name, group in grouped:
    # if count > n_rows:
    #     break
    if len(group) < 150:
        continue

    max_value = group['y'].max()
    if IS_DEBUG:
        print(max_value)
        print(name)
        print(group)
    test = group

    test['cap'] = max_value
    test['floor'] = 0
    model = Prophet(growth = 'logistic', daily_seasonality = False, weekly_seasonality = False,
                    yearly_seasonality =  False)
    # fit the model
    model.fit(test)

    future = model.make_future_dataframe(periods=1826)
    future['cap'] = max_value
    future['floor'] = 0

    # use the model to make a forecast
    forecast = model.predict(future)
    # summarize the forecast
    if IS_DEBUG:
        print("forecast:")
        print(forecast[['ds', 'yhat']].head())
    # plot forecast
    # model.plot(forecast)

    fig = plot_plotly(model, forecast)  # This returns a plotly Figure
    py.iplot(fig)

    # Data generation for analysis
    data = test.y
    trend, h, p, z, Tau, s, var_s, slope, intercept = mk.original_test(data)

    mk_res.append([name, trend, h, p, z, Tau, s, var_s, slope, intercept])

    count+=1


mk_df = pd.DataFrame(mk_res, columns=['Well name', 'trend', 'h', 'p', 'z', 'Tau', 's', 'var_s','slope' , 'intercept'])
mk_df

